In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/MIT project

/content/drive/MyDrive/MIT project


In [ ]:
!pip install periodictable
!pip install torch
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas

In [ ]:
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import os
# from classes import *
# from exclude import *
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import re
from collections import defaultdict
import copy
import matplotlib as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
# Complete periodic table with element positions (index starts from 1)
periodic_table = {
    'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9, 'Ne': 10,
    'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17, 'Ar': 18, 'K': 19, 'Ca': 20,
    'Sc': 21, 'Ti': 22, 'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26, 'Co': 27, 'Ni': 28, 'Cu': 29, 'Zn': 30,
    'Ga': 31, 'Ge': 32, 'As': 33, 'Se': 34, 'Br': 35, 'Kr': 36, 'Rb': 37, 'Sr': 38, 'Y': 39, 'Zr': 40,
    'Nb': 41, 'Mo': 42, 'Tc': 43, 'Ru': 44, 'Rh': 45, 'Pd': 46, 'Ag': 47, 'Cd': 48, 'In': 49, 'Sn': 50,
    'Sb': 51, 'Te': 52, 'I': 53, 'Xe': 54, 'Cs': 55, 'Ba': 56, 'La': 57, 'Ce': 58, 'Pr': 59, 'Nd': 60,
    'Pm': 61, 'Sm': 62, 'Eu': 63, 'Gd': 64, 'Tb': 65, 'Dy': 66, 'Ho': 67, 'Er': 68, 'Tm': 69, 'Yb': 70,
    'Lu': 71, 'Hf': 72, 'Ta': 73, 'W': 74, 'Re': 75, 'Os': 76, 'Ir': 77, 'Pt': 78, 'Au': 79, 'Hg': 80,
    'Tl': 81, 'Pb': 82, 'Bi': 83, 'Po': 84, 'At': 85, 'Rn': 86, 'Fr': 87, 'Ra': 88, 'Ac': 89, 'Th': 90,
    'Pa': 91, 'U': 92, 'Np': 93, 'Pu': 94, 'Am': 95, 'Cm': 96, 'Bk': 97, 'Cf': 98, 'Es': 99, 'Fm': 100,
    'Md': 101, 'No': 102, 'Lr': 103, 'Rf': 104, 'Db': 105, 'Sg': 106, 'Bh': 107, 'Hs': 108, 'Mt': 109,
    'Ds': 110, 'Rg': 111, 'Cn': 112, 'Nh': 113, 'Fl': 114, 'Mc': 115, 'Lv': 116, 'Ts': 117, 'Og': 118
}

In [ ]:
# Function to parse the chemical formula
def parse_formula(formula):
    # Pattern to match elements, counts, and groups with parentheses
    #pattern = r'([A-Z][a-z]?|\([^\(\)]+\))(\d*)'
    # new pattern to accept non integer numbers
    pattern = r'([A-Z][a-z]?|\([^\(\)]+\))(\d*\.?\d*)'
    matches = re.findall(pattern, formula)
    composition = defaultdict(float)

    def add_composition(comp_dict):
        for element, count in matches:
            if count == '':
                count = 1
            else:
                count = float(count)
            if element.startswith('('):
                # Remove parentheses and parse the inner formula
                inner_formula = element[1:-1]
                inner_composition = parse_formula(inner_formula)

                for inner_element, inner_count in inner_composition.items():
                    comp_dict[inner_element] += inner_count * count
            else:
                comp_dict[element] += count

    add_composition(composition)

    return composition

def parse_prettyformula(formula):
    pattern = r'([A-Z][a-z]?)(\d*)'
    matches = re.findall(pattern, formula)
    composition=defaultdict(int)
    for element, count in matches:
        if count=='' or count == None:
            count=1
        composition[element]= int(count)
    return composition


# Function to create the compositional vector
def encode_compositional_vector(formula):
    composition = parse_formula(formula)
    total_atoms = sum(composition.values())
    vector = [0] * len(periodic_table)  # Create a zero vector with length equal to the number of elements

    for element, count in composition.items():
        index = periodic_table[element]
        vector[index - 1] = count / total_atoms  # Convert to fraction

    return vector


# input: list of materials
# output: list of compositional vectors
def convertFormulaToVectors(formula:list):
    output=list()
    for material in formula:
        output.append(encode_compositional_vector(material))
    return output

# concatenates precursors and targets to a 1 dimensional array
#input: list of compositional vectors
def concatenate(precursors:list, targets:list):
    maxPrecursors= 6
    maxTargets=6
    vectorLength=118

    precursors.append((maxPrecursors-len(precursors))*(vectorLength*[0]))
    targets.append((maxTargets-len(targets))*(vectorLength*[0]))

    return np.concatenate([np.concatenate(precursors),np.concatenate(targets)])

#input: list of compositional vectors
# output: ML ready summed up version
def sumCompositional(precursors:list, targets:list):
    prec=list()
    for i in range(len(precursors[0])):
        num=0
        for k in range(len(precursors)):
            num+=precursors[k][i]
        prec.append(num)

    targ=list()
    for i in range(len(targets[0])):
        num=0
        for k in range(len(targets)):
            num+=targets[k][i]
        targ.append(num)

    return np.concatenate([prec,targ])


#input: lhs and rhs to make it ML ready
def transform(leftSide:str, rightSide:str, concat=False):
    leftSide.replace("[","(")
    leftSide.replace("]",")")
    leftSide.replace("·","+")
    rightSide.replace("[","(")
    rightSide.replace("]",")")
    rightSide.replace("·","+")

    precursors=leftSide.split("+")
    precursors=list(map(str.strip, precursors))

    targets=rightSide.split("+")
    targets=list(map(str.strip, targets))

    precVectors=convertFormulaToVectors(precursors)
    targVectors=convertFormulaToVectors(targets)

    #concatenate or sum
    if concat:
        LMready=concatenate(precVectors,targVectors)
    else:
        LMready=sumCompositional(precVectors,targVectors)

    return LMready



In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(236, 360)
        self.fc2 = nn.Linear(360, 180)
        self.fc3 = nn.Linear(180, 100)
        self.fc4 = nn.Linear(100, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x
model = NN()

In [ ]:
path = os.getcwd()
df = pd.read_excel(path + '/data/reaction_entries.xlsx')


KeyError: 'lhs'

In [ ]:
TASK = 'calc'

lhs = df['LHS']
rhs = df['RHS']

calc_temp = df['calcination_temp']
sintering_temp = df['sintering_temp']

calc_temp_nonna = calc_temp.dropna()
sintering_temp_nonna = sintering_temp.dropna()

bool_calc_temp = calc_temp.notna()
bool_sintering_temp = sintering_temp.notna()

lhs_actual_calc = df.loc[bool_calc_temp, 'LHS']
rhs_actual_calc = df.loc[bool_calc_temp, 'RHS']
indexes_non_nan_calc = df.index[bool_calc_temp]

lhs_actual_sint = df.loc[bool_sintering_temp, 'LHS']
rhs_actual_sint = df.loc[bool_sintering_temp, 'RHS']
indexes_non_nan_sint = df.index[bool_sintering_temp]

X = []
weird = []
temp_mask = []
if TASK.lower()[0] == 'c':
    lhs_calc = list(lhs_actual_calc)
    rhs_calc = list(rhs_actual_calc)
    for i in range(len(rhs_calc)):
      try:
        x = transform(lhs_calc[i], rhs_calc[i])
        X.append(x)
        temp_mask.append(True)
      except:
        string = f"lhs:{lhs_calc[i]}, rhs:{rhs_calc[i]}"
        weird.append(string)
        temp_mask.append(False)
        continue
    X = torch.tensor(X)
    y = torch.tensor(list(calc_temp_nonna))
    y = y[temp_mask]#Filter out the temps of weird elements
else:
    lhs_sint = list(lhs_actual_sint)
    rhs_sint = list(rhs_actual_sint)
    for i in range(len(rhs_sint)):
      try:
        x = transform(lhs_sint[i], rhs_sint[i])
        X.append(x)
        temp_mask.append(True)
      except:
        string = f"lhs:{lhs_sint[i]}, rhs:{rhs_sint[i]}"
        weird.append(string)
        temp_mask.append(False)
        continue
    X = torch.tensor(X)
    y = torch.tensor(list(sintering_temp_nonna))
    y = y[temp_mask]#Filter out the temps of weird elements

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
data_list = TensorDataset(X, y)

print(X.shape)
print(y.shape)
print(len(weird))

torch.Size([4765, 236])
torch.Size([4765])
453


<ipython-input-91-266a8d1530c3>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)  # Ensure dtype is float32
<ipython-input-91-266a8d1530c3>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [ ]:
def train(train_loader, device, model, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    return average_loss

def validate(validation_loader, device, model, criterion):
    model.eval()
    running_loss = 0.0
    validation_outputs = []
    validation_truth = []

    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            validation_outputs.append(outputs.detach().numpy())
            validation_truth.append(labels.detach().numpy())

    average_loss = running_loss / len(validation_loader)
    validation_outputs = np.concatenate(validation_outputs)
    validation_truth = np.concatenate(validation_truth)

    return average_loss, validation_outputs, validation_truth


def test(test_loader, device, model, criterion):
    model.eval()
    running_loss = 0.0
    test_outputs = []
    test_truth = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            test_outputs.append(outputs.detach().numpy())
            test_truth.append(targets.detach().numpy())

    average_loss = running_loss / len(test_loader)
    test_outputs = np.concatenate(test_outputs)
    test_truth = np.concatenate(test_truth)

    return average_loss, test_outputs, test_truth

In [ ]:
EPOCHS = 10
learning_rate = 0.1
sampling_size = 6

validation_losses = []
training_losses = []

best_vals =[]
best_models = []
best_tests = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_test_split_index = int(0.9 * len(data_list))

train_val_data = Subset(data_list, range(train_test_split_index))
test_data = Subset(data_list, range(train_test_split_index, len(data_list)))

test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

data_len = len(train_val_data)

num_batches = data_len // sampling_size

indices = np.arange(data_len)

for i in range(num_batches):
    best_val_loss = np.inf

    start_index = i * sampling_size

    train_indices = np.setdiff1d(indices, val_indices)
    val_indices = indices[start_index:start_index + sampling_size]
    train_data = Subset(train_val_data, train_indices)
    val_data = Subset(train_val_data, val_indices)

    print(f"Batch {i+1}/{num_batches}, Train Size: {len(train_data)}, Validation Size: {len(val_data)}")

    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    validation_loader = DataLoader(val_data, batch_size=16, shuffle=False)

    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                        factor=0.8, patience=5,
                                                        min_lr=0.0000001)
    criterion = nn.L1Loss()
    best_validation_loss = float('inf')

    for epoch in range(1, EPOCHS+1):

        model.train()

        train_loss = train(train_loader, device, model, optimizer, criterion)
        scheduler.step(train_loss)

        loss = train(train_loader, device, model, optimizer, criterion)
        scheduler.step(loss)

        validation_loss, validation_output, validation_truth_temp = validate(validation_loader, device, model, criterion)
        # print(f"Epoch {epoch+1}, Train Loss: {loss:.4f}, Validation Loss: {validation_loss:.4f}")
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            best_model_state = copy.deepcopy(model.state_dict())
            best_model = model
            best_val_loss = validation_loss
            print(f"Best Validation Loss: {best_val_loss:.4f}")

        # Store the best validation loss for the current epoch
        validation_losses.append(best_val_loss)
        training_losses.append(train_loss)

model.load_state_dict(best_model_state)
test_loss, test_outputs, test_truth = test(test_loader, device, model, criterion)
print(f"Test Loss: {test_loss:.4f}")
print(f"validation losses: {validation_losses}")


Batch 1/714, Train Size: 4282, Validation Size: 6
loss: 210.1541013575312
Best Validation Loss: 279.1457


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


loss: 210.81513253966375
Best Validation Loss: 262.1667
loss: 210.80426515038334
loss: 209.4438491081124
loss: 209.45552299271768
loss: 209.01522901164952
loss: 209.86683851213598
loss: 209.3789345470827
loss: 209.16754116229157
loss: 209.4639983675373
Batch 2/714, Train Size: 4282, Validation Size: 6
loss: 209.55316529345157
Best Validation Loss: 226.5285
loss: 208.12707912388132
loss: 211.48283258124962
loss: 207.90568212252944


KeyboardInterrupt: 

In [ ]:
# Plotting validation loss and training loss vs. epochs
plt.figure(figsize=(12, 6))

plt.plot(range(1, len(training_losses) + 1), training_losses, marker='o', linestyle='-', color='r', label='Training Loss')
plt.plot(range(1, len(validation_losses) + 1), validation_losses, marker='o', linestyle='-', color='b', label='Validation Loss')

plt.title('Training and Validation Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
X = np.random.rand(1000, 119)  # Example: 1000 samples, 10 features
y = np.random.rand(1000)      # Example: 1000 target values


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

evals = [(dtrain, 'train'), (dval, 'eval')]

num_boost_round = 100
early_stopping_rounds = 10
model = xgb.train(params, dtrain, num_boost_round, evals, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

y_pred = model.predict(dtest)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Test RMSE: {test_rmse:.4f}')

In [41]:
import optuna
import sqlite3

# Define the storage URL for Optuna
storage_url = 'sqlite:////home/students/code/MLP/Menelaos/reaction_conditions/MLP/storage/db_9_4_22_1.sqlite3'

# Get the list of Optuna study summaries
study_summaries = optuna.study.get_all_study_summaries(storage=storage_url)

# Extract the study names from the summaries
study_names = [summary.study_name for summary in study_summaries]

# Print the study names
print(study_names)


# Get the hyperparameters for the specified study names
study_hyperparams = []
for study_name in [study_names[0], study_names[15], study_names[30], study_names[45], study_names[60]]:
    study = optuna.study.load_study(study_name, storage=storage_url)
    study_hyperparams.append(study.best_params)

print(study_hyperparams)

["{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0", "{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=1", "{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=2", "{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=3", "{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=4", "{'featurisation': 'mtencoder_512', 'model_type': 'NN', 'task': 'sint', 'use_only_target': False, 'concat': False}_split=0", "{'featurisation': 'mtencoder_512', 'model_type': 'NN', 'task': 'sint', 'use_only_target': False, 'concat': False}_split=1", "{'featurisation': 'mtencoder_512', 'model_type': 'NN', 'task': 'sint', 'use_only_target': False, 'concat': False}_split=2", "{'featurisation': 'mtencoder_512', 'model_type': 'NN', 'task': 'sint', 'use_on

/tmp/ipykernel_63160/3308839258.py:20: FutureWarning: load_study() got {'study_name'} as positional arguments but they were expected to be given as keyword arguments.
  study = optuna.study.load_study(study_name, storage=storage_url)


In [40]:
[study_names[0], study_names[15], study_names[30], study_names[45], study_names[60]]

["{'featurisation': 'mtencoder_512', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0",
 "{'featurisation': 'mtencoder_256', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0",
 "{'featurisation': 'composition', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0",
 "{'featurisation': 'matscibert', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0",
 "{'featurisation': 'matminer', 'model_type': 'GNN', 'task': 'sint', 'use_only_target': False}_split=0"]

In [42]:
study_hyperparams

[{'epochs': 50,
  'base_layer_dim_log': 10,
  'batch_size': 6,
  'nn_layers': 4,
  'n_layers': 1,
  'conv_steps': 4},
 {'epochs': 33,
  'base_layer_dim_log': 10,
  'batch_size': 6,
  'nn_layers': 4,
  'n_layers': 2,
  'conv_steps': 4},
 {'epochs': 39,
  'base_layer_dim_log': 9,
  'batch_size': 4,
  'nn_layers': 4,
  'n_layers': 3,
  'conv_steps': 2},
 {'epochs': 48,
  'base_layer_dim_log': 10,
  'batch_size': 4,
  'nn_layers': 2,
  'n_layers': 3,
  'conv_steps': 3},
 {'epochs': 33,
  'base_layer_dim_log': 10,
  'batch_size': 4,
  'nn_layers': 2,
  'n_layers': 4,
  'conv_steps': 3}]